In [2]:
# Suprimir a exibição de mensagens de aviso (warnings).
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

In [4]:
# Quantidade de vinhos que serão recomendados
num_recomenda = 5

In [5]:
# Carregar o dataset de vinhos
file_path = 'dados_csv//xwines//XWines_Slim_1K_wines.csv'
data = pd.read_csv(file_path)

# Carregar o dataset de avaliações
ratings_file_path = 'dados_csv//xwines//XWines_Slim_150K_ratings.csv'
ratings_data = pd.read_csv(ratings_file_path)

# Criar a matriz de avaliações
R_df = ratings_data.pivot(index='UserID', columns='WineID', values='Rating')

In [6]:
# Pré-processamento básico

# Define todas as entradas iguais a zero para NaN (não um número)
R = R_df.fillna(0).values

# Configurar opções de impressão para exibir todas as linhas da matriz de avaliações
np.set_printoptions(threshold=np.inf)

# Normalizar as avaliações subtraindo a média de cada usuário para lidar com o viés do usuário
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [7]:
# Plotar os resultados da normalização no plano cartesiado
'''
U, S, VT = randomized_svd(R_demeaned, n_components=2)
resultados_svd = U.dot(np.diag(S))
print(resultados_svd)

plt.scatter(resultados_svd[:, 0], resultados_svd[:, 1], cmap='viridis')
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.title('Visualização de Dados Reduzidos com SVD')
plt.show()
'''

"\nU, S, VT = randomized_svd(R_demeaned, n_components=2)\nresultados_svd = U.dot(np.diag(S))\nprint(resultados_svd)\n\nplt.scatter(resultados_svd[:, 0], resultados_svd[:, 1], cmap='viridis')\nplt.xlabel('Dimensão 1')\nplt.ylabel('Dimensão 2')\nplt.title('Visualização de Dados Reduzidos com SVD')\nplt.show()\n"

In [8]:
# A seguir, calculamos a variância explicada acumulada para diferentes quantidades de 
# dimensões (componentes principais). Observe que, como esperado, a variância explicada 
# acumulada aumenta à medida que adicionamos mais dimensões. Você pode escolher o número 
# de dimensões que captura uma porcentagem significativa # da variância total. Por exemplo, 
# XWines_Test_100_wines.csv tem 100 vinhos (100 dimensões), se 90% da variância é explicada 
# por apenas 77 vinhos (dimensões) e isto é considerado adequado para a resolução do problema,
# então você pode usar 77 dimensões para a redução de dimensionalidade.
'''
variance_explained = []
for n in range(1, R_demeaned.shape[1] + 1):
    svd = TruncatedSVD(n_components=n)
    X_svd = svd.fit_transform(R_demeaned)
    variance_explained.append(np.sum(svd.explained_variance_ratio_))
    print(f'Para n={n}, variance_explained={variance_explained[n-1]}')
'''

"\nvariance_explained = []\nfor n in range(1, R_demeaned.shape[1] + 1):\n    svd = TruncatedSVD(n_components=n)\n    X_svd = svd.fit_transform(R_demeaned)\n    variance_explained.append(np.sum(svd.explained_variance_ratio_))\n    print(f'Para n={n}, variance_explained={variance_explained[n-1]}')\n"

In [9]:
# Pré-processamento: redução de dimensionalidade

# Aplicar SVD para decompor (fatorar) a matriz de avaliações
U, sigma, Vt = randomized_svd(R_demeaned, n_components=77)

In [10]:
# Reconstruir a matriz de avaliações previstas considerando o viés do usário (user_ratings_mean)
predicted_ratings = np.dot(np.dot(U, np.diag(sigma)), Vt) + user_ratings_mean.reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=R_df.columns)

In [11]:
# Função para recomendar vinhos com base em harmonização
def recommend_wines(user_id, food, num_recommendations=num_recomenda):
    # Obter o número da linha referente ao usuário na matriz de avaliações
    user_row_number = R_df.index.get_loc(user_id)

    # Avaliações originais feitas pelo usuário em ordem decrescente (há vinhos não avaliados)
    #original_user_ratings = R_df.iloc[user_row_number].sort_values(ascending=False)

    # Obter as avaliações previstas para o usuário em ordem decrescente (todos os vinhos contém avaliações)
    sorted_user_ratings = predicted_ratings_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Filtrar os vinhos que harmonizam com a comida selecionada
    harmonized_wines = data[data['Harmonize'].str.contains(food, case=False, na=False)]

    # Criar DataFrame para armazenar os vinhos de harmonized_wines (vinhos que harmonizam 
    # com a comida) e que estão em sorted_user_ratings (avaliações previstas para o usuário 
    # em ordem decrescente)
    merged_data = pd.DataFrame()

    # Iterar pelos WineID em sorted_user_ratings
    for wine_id in sorted_user_ratings.index:
        # Verificar se o WineID está em harmonized_wines
        if wine_id in harmonized_wines['WineID'].values:
            # Combinar os DataFrames usando merge
            merged_data = pd.concat([merged_data, harmonized_wines[harmonized_wines['WineID'] == wine_id]])
    # Agora, merged_data contém todos os dados de harmonized_wines para os WineID presentes em sorted_user_ratings
    #print(merged_data)

    # Recomendar os vinhos com as maiores avaliações previstas que harmonizam com a comida
    recommended_wines = merged_data[['WineName', 'Harmonize']].head(num_recommendations) 
    #recommended_wines = merged_data[['WineID', 'WineName', 'Type', 'Grapes', 'Harmonize']].head(num_recommendations) 

    return recommended_wines

In [12]:
# Exemplo de uso da função
user_id = 1258705  # Substitua pelo ID do usuário desejado 1003868 1258705
food = 'Beef'  # Substitua pela comida desejada
recommended_wines = recommend_wines(user_id, food, num_recomenda)
print("Vinhos recomendados: ")
print(recommended_wines)
print()

Vinhos recomendados: 
                   WineName                                 Harmonize
374  Brunello di Montalcino  ['Beef', 'Lamb', 'Game Meat', 'Poultry']
54        Monte Velho Tinto       ['Beef', 'Lamb', 'Veal', 'Poultry']
640        Trumpeter Malbec               ['Beef', 'Lamb', 'Poultry']
650          Reserva Malbec               ['Beef', 'Lamb', 'Poultry']
58     Flor de Crasto Tinto    ['Beef', 'Pasta', 'Lamb', 'Game Meat']

